In [1]:
import pandas as pd
import math
import torch
import torch.nn as nn
import torch.nn.functional as F
from pytorch_metric_learning import losses

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
import torch
import pandas as pd
import numpy as np
from torch.utils.data import DataLoader, Dataset
from sklearn.metrics import classification_report
from sklearn import metrics
import os
import torch.nn as nn

X=pd.read_excel('mimic/'+str(3)+'_mimictrainEncoded.xlsx').drop(columns=['Unnamed: 0'])
val=pd.read_excel('mimic/'+str(3)+'_mimicvalidationEncoded.xlsx').reindex(columns=X.columns)
vlabel=pd.read_csv('mimic/'+str(3)+'_mimicvalidationlabel.csv')['0']
y=pd.read_csv('mimic/'+str(3)+'_mimictrainlabel.csv')['0']
testlabel=pd.read_csv('mimic/'+str(3)+'_mimictestlabel.csv')['0']
test=pd.read_excel('mimic/'+str(3)+'_mimictestEncoded.xlsx').reindex(columns=X.columns)
class Dataset(Dataset):
    def __init__(self, x, y):
#         reader = pd.read_excel(fname)
        all_data = []
        for i in range(x.shape[0]):
            feature = torch.tensor(x.iloc[i].to_list(),dtype=torch.float32).to('cpu')
            class_n = torch.tensor(y[i],dtype=torch.long).to('cpu')
            data = {
                'features': feature,
                'class_n': int(class_n),
            }
            all_data.append(data)
        self.data = all_data
        print('finished data prepration')

    def __getitem__(self, index):
        return self.data[index]

    def __len__(self):
        return len(self.data)
def _reset_params(model, initializer):
    for child in model.children():
        for p in child.parameters():
            if p.requires_grad:
                if initializer == 'uniform':
                    stdv = 1. / math.sqrt(p.shape[0])
                    torch.nn.init.uniform_(p, a=-stdv, b=stdv)
                elif initializer == 'normal':
                    torch.nn.init.normal_(p)
                elif initializer == 'dirac':
                    torch.nn.init.dirac_(p, groups=1)
                elif initializer == 'orthogonal':
                    torch.nn.init.orthogonal_(p,gain=1)
                elif initializer =='kaiming':
                    torch.nn.init.kaiming_uniform_(p,a=0,mode='fan_in',nonlinearity='leaky_relu')
                elif initializer == 'ones':
                    torch.nn.init.ones_(p)
                elif initializer=='xavier_normal':
                    torch.nn.init.xavier_normal_(p,gain=1.0)
train = X
val = val
test = test

"""Here you need input your real label, the lables shown here are just test the model"""
train_y = np.array(y)
val_y =  np.array(vlabel)
test_y = np.array(testlabel)


train = Dataset(train, train_y)
val = Dataset(val, val_y)
test = Dataset(test, test_y)

# Load training dataset labels
temp = [t['class_n'] for t in train]
# Count different classes sample number
class_sample_count = np.array([len(np.where(temp == t)[0]) for t in np.unique(temp)])
# Calculate weights for different classes
weight_temp = 1. / class_sample_count
# Apply weights to each sample
samples_weight = np.array([weight_temp[t] for t in temp])
samples_weight = torch.from_numpy(samples_weight)
# Build sampler for dataloader
sampler = torch.utils.data.sampler.WeightedRandomSampler(samples_weight.type('torch.DoubleTensor'), len(samples_weight))

train_x = DataLoader(dataset=train, batch_size=64, sampler = sampler)
val_x = DataLoader(dataset=val, batch_size=64, shuffle=False)
test_x = DataLoader(dataset=test, batch_size=64, shuffle=False)

def _evaluate_acc_f1(data_loader, model):
    n_correct, n_total = 0, 0
    t_targets_all, t_outputs_all = None, None
    # switch model to evaluation mode
    model.eval()
    with torch.no_grad():
        for t_batch, t_sample_batched in enumerate(data_loader):
            t_inputs = t_sample_batched['features']
            t_targets = t_sample_batched['class_n']
            t_outputs = model(t_inputs)
#             m=torch.nn.Softmax(dim=1)#softmax layer
#             output_new=m(t_outputs)#softmax output
#             print(output_new)
#             print(sum(t_targets))
#             threshold=sorted(output_new.numpy(),reverse=True,key=lambda x:x[1])[sum(t_targets)][1]#possibly threshold?
#             print('t_outputs',t_outputs)
#             print('output_new',output_new)
#             print(threshold)
            #如果要在validation里用到 threshold，我不太确定该怎么改下面的argmax function
            n_correct += (torch.argmax(t_outputs, -1) == t_targets).sum().item()#to change
            #swap
            n_total += len(t_outputs)

            if t_targets_all is None:
                t_targets_all = t_targets
                t_outputs_all = t_outputs
            else:
                t_targets_all = torch.cat((t_targets_all, t_targets), dim=0)
                t_outputs_all = torch.cat((t_outputs_all, t_outputs), dim=0)

    acc = n_correct / n_total
    f1 = metrics.f1_score(t_targets_all.cpu(), torch.argmax(t_outputs_all, -1).cpu(), labels=[0, 1], average='macro')
    report = classification_report(t_targets_all.cpu(), torch.argmax(t_outputs_all, -1).cpu(), target_names=['class0','class1'], output_dict=True, digits = 4)
    return acc, f1, report



# N is batch size; D_in is input dimension;
# H is hidden dimension; D_out is output dimension.

"""You should test different Hidden dimension, and learning rate to find the best model"""
N, D_in, H, D_out = 64, 15, 5, 2
model = torch.nn.Sequential(
    torch.nn.Linear(D_in, H),
    torch.nn.ReLU(),
    torch.nn.Linear(H, D_out),
)

_reset_params(model, 'uniform')
#===================================================================================#
epoch = 100
loss_fn = torch.nn.CrossEntropyLoss()
# learning_rate = 1e-4
learning_rate=4e-3
# learning_rate=2e-2
model.train()
global_step = 0
max_val_f1 = 0
n_correct, n_total, loss_total = 0, 0, 0
t_targets_all, t_outputs_all = None, None

#===================================================================================#

mimicloss=[]
for t in range(epoch):
    for i_batch, sample_batched in enumerate(train_x):        
        outputs = model(sample_batched['features'])
        targets = sample_batched['class_n']
        m=torch.nn.Softmax(dim=1)
        output_new=m(outputs)
        threshold=sorted(output_new.detach().numpy(),reverse=True,key=lambda x:x[1])[sum(targets)][1]
        print("threshold is ",threshold)
        #不太确定如何modify 有threshold 的output 然后放到loss function里面
        loss = loss_fn(outputs, targets)     
        model.zero_grad()
        loss.backward()
        
        if t_targets_all is None:
            t_targets_all = targets
            t_outputs_all = outputs
        else:
            t_targets_all = torch.cat((t_targets_all, targets), dim=0)
            t_outputs_all = torch.cat((t_outputs_all, outputs), dim=0)
            
#         report = classification_report(t_targets_all.cpu(), torch.argmax(t_outputs_all, -1).cpu(), target_names=['class0','class1'], output_dict=True, digits = 4)
        
        n_correct += (torch.argmax(outputs, -1) == targets).sum().item()
        n_total += len(outputs)
        loss_total += loss.item() * len(outputs)
        if global_step % 2 == 0:
            train_acc = n_correct / n_total
            train_loss = loss_total / n_total
            mimicloss.append(train_loss)
            print('loss: {:.4f}, acc: {:.4f}'.format(train_loss, train_acc))
        with torch.no_grad():
            for param in model.parameters():
                param -= learning_rate * param.grad
    #=============================================================
    #validation
    val_acc, val_f1, report_val  = _evaluate_acc_f1(val_x, model)
    print('val_report: {}'.format(report_val['class1']))
    if report_val['class1']['f1-score'] > max_val_f1:
                max_val_f1 = report_val['class1']['f1-score']

                if not os.path.exists('state_dict/'+'mimic'):
                    os.mkdir('state_dict/'+'mimic')
                path = 'state_dict/'+'mimic'+'/{0}_{1}_val_f1_{2}'.format('neuralNet', 'mimic', round(report_val['class1']['f1-score'], 4))
                torch.save(model.state_dict(), path)        

#predict on test
best_path = path

model.load_state_dict(torch.load(best_path,map_location=torch.device('cpu')))
model.eval()
test_acc, test_f1, report_test  = _evaluate_acc_f1(test_x, model)

report_test

finished data prepration
finished data prepration
finished data prepration
threshold is  1.0
loss: 78.2844, acc: 0.4844
threshold is  0.03009596
loss: 45.7394, acc: 0.5391
threshold is  0.00075534807
loss: 33.4351, acc: 0.5521
threshold is  0.0050551607
loss: 26.5180, acc: 0.5859
threshold is  0.05065493
loss: 21.8832, acc: 0.6125
threshold is  0.012673633
loss: 19.2395, acc: 0.6042
threshold is  0.097839415
loss: 17.0112, acc: 0.5982
threshold is  0.17632195
loss: 15.3493, acc: 0.5898
threshold is  0.29150602
loss: 13.7524, acc: 0.5938
threshold is  0.29186833
loss: 12.5347, acc: 0.5922
threshold is  0.29206628
loss: 11.5014, acc: 0.5838
threshold is  0.2925461
loss: 10.6137, acc: 0.5781
threshold is  0.29292634
loss: 9.8476, acc: 0.5841
threshold is  0.29311207
loss: 9.1928, acc: 0.5837
threshold is  0.29340777
loss: 8.6307, acc: 0.5781
threshold is  0.2938482
loss: 8.1349, acc: 0.5762
threshold is  0.29419428
loss: 7.7009, acc: 0.5708
threshold is  0.29462683
loss: 7.3176, acc: 0.56

threshold is  0.35369718
loss: 1.3720, acc: 0.5068
threshold is  0.35405
loss: 1.3682, acc: 0.5070
threshold is  0.35431483
loss: 1.3644, acc: 0.5071
threshold is  0.35461247
loss: 1.3607, acc: 0.5071
threshold is  0.35496745
loss: 1.3571, acc: 0.5069
threshold is  0.3553271
loss: 1.3538, acc: 0.5066
threshold is  0.3557332
loss: 1.3506, acc: 0.5062
threshold is  0.35619935
loss: 1.3475, acc: 0.5058
threshold is  0.35664982
loss: 1.3441, acc: 0.5056
threshold is  0.35701272
loss: 1.3406, acc: 0.5056
threshold is  0.35735106
loss: 1.3374, acc: 0.5056
threshold is  0.35767445
loss: 1.3339, acc: 0.5059
threshold is  0.3579022
loss: 1.3307, acc: 0.5057
threshold is  0.3582828
loss: 1.3271, acc: 0.5061
threshold is  0.35844842
loss: 1.3236, acc: 0.5064
threshold is  0.3586881
loss: 1.3206, acc: 0.5063
threshold is  0.35901204
loss: 1.3173, acc: 0.5064
threshold is  0.35931405
loss: 1.3143, acc: 0.5062
threshold is  0.35963744
loss: 1.3113, acc: 0.5061
threshold is  0.36002505
loss: 1.3079, 

threshold is  0.4133869
loss: 0.9995, acc: 0.5040
threshold is  0.41363683
loss: 0.9986, acc: 0.5040
threshold is  0.4138354
loss: 0.9977, acc: 0.5039
threshold is  0.41420034
loss: 0.9970, acc: 0.5037
threshold is  0.41455618
loss: 0.9960, acc: 0.5037
threshold is  0.4147789
loss: 0.9952, acc: 0.5038
threshold is  0.41495922
loss: 0.9942, acc: 0.5038
threshold is  0.41528478
loss: 0.9934, acc: 0.5035
threshold is  0.41575193
loss: 0.9925, acc: 0.5038
threshold is  0.41579473
loss: 0.9917, acc: 0.5035
threshold is  0.4161987
loss: 0.9909, acc: 0.5035
threshold is  0.41643134
loss: 0.9901, acc: 0.5033
threshold is  0.41681245
loss: 0.9894, acc: 0.5031
threshold is  0.41726553
loss: 0.9886, acc: 0.5029
threshold is  0.41757795
loss: 0.9879, acc: 0.5028
threshold is  0.4178524
loss: 0.9871, acc: 0.5027
threshold is  0.41823292
loss: 0.9862, acc: 0.5027
threshold is  0.41847628
loss: 0.9853, acc: 0.5028
threshold is  0.41863632
loss: 0.9845, acc: 0.5029
threshold is  0.4188326
loss: 0.9836

loss: 0.8818, acc: 0.4996
threshold is  0.45853975
loss: 0.8814, acc: 0.4996
threshold is  0.45874637
loss: 0.8810, acc: 0.4995
threshold is  0.45897236
loss: 0.8807, acc: 0.4993
threshold is  0.4594244
loss: 0.8803, acc: 0.4993
threshold is  0.459523
loss: 0.8800, acc: 0.4992
threshold is  0.45980135
loss: 0.8795, acc: 0.4992
threshold is  0.46001422
loss: 0.8791, acc: 0.4992
threshold is  0.46013877
loss: 0.8787, acc: 0.4990
threshold is  0.4605523
loss: 0.8784, acc: 0.4989
threshold is  0.46086
loss: 0.8779, acc: 0.4989
threshold is  0.46102834
loss: 0.8776, acc: 0.4988
threshold is  0.46126264
loss: 0.8772, acc: 0.4989
threshold is  0.46133187
loss: 0.8768, acc: 0.4989
threshold is  0.4614896
loss: 0.8764, acc: 0.4990
threshold is  0.4613913
loss: 0.8761, acc: 0.4990
threshold is  0.4616161
loss: 0.8756, acc: 0.4991
threshold is  0.4617021
loss: 0.8752, acc: 0.4990
threshold is  0.46200752
loss: 0.8748, acc: 0.4991
threshold is  0.4619984
loss: 0.8744, acc: 0.4993
threshold is  0.4

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

...

loss: 0.6411, acc: 0.5953
threshold is  0.5650272
loss: 0.6411, acc: 0.5953
threshold is  0.56498885
loss: 0.6411, acc: 0.5953
threshold is  0.5649746
loss: 0.6411, acc: 0.5953
threshold is  0.56500226
loss: 0.6410, acc: 0.5953
threshold is  0.56496525
loss: 0.6410, acc: 0.5953
threshold is  0.5648336
loss: 0.6410, acc: 0.5953
threshold is  0.56484437
loss: 0.6410, acc: 0.5953
threshold is  0.5647238
loss: 0.6410, acc: 0.5953
threshold is  0.56471676
loss: 0.6410, acc: 0.5953
threshold is  0.5647454
loss: 0.6410, acc: 0.5953
threshold is  0.5647069
loss: 0.6410, acc: 0.5953
threshold is  0.5646404
loss: 0.6410, acc: 0.5953
threshold is  0.56462896
loss: 0.6410, acc: 0.5953
threshold is  0.56457704
loss: 0.6410, acc: 0.5953
threshold is  0.56462663
loss: 0.6410, acc: 0.5953
threshold is  0.5647375
loss: 0.6410, acc: 0.5953
threshold is  0.56468487
loss: 0.6410, acc: 0.5953
threshold is  0.5646367
loss: 0.6410, acc: 0.5953
threshold is  0.5647554
loss: 0.6410, acc: 0.5953
threshold is  0

loss: 0.6395, acc: 0.5962
threshold is  0.56486934
loss: 0.6395, acc: 0.5962
threshold is  0.5647184
loss: 0.6395, acc: 0.5962
threshold is  0.56468624
loss: 0.6395, acc: 0.5962
threshold is  0.5648159
loss: 0.6395, acc: 0.5962
threshold is  0.5649961
loss: 0.6394, acc: 0.5962
threshold is  0.5649049
loss: 0.6395, acc: 0.5962
threshold is  0.56466085
loss: 0.6394, acc: 0.5962
threshold is  0.56465954
loss: 0.6394, acc: 0.5962
threshold is  0.5646347
loss: 0.6394, acc: 0.5962
threshold is  0.5645747
loss: 0.6394, acc: 0.5962
threshold is  0.56456697
loss: 0.6394, acc: 0.5962
threshold is  0.56444937
loss: 0.6394, acc: 0.5962
threshold is  0.56449133
loss: 0.6394, acc: 0.5962
threshold is  0.56460273
loss: 0.6394, acc: 0.5962
threshold is  0.5647697
loss: 0.6394, acc: 0.5962
threshold is  0.56490314
loss: 0.6394, acc: 0.5962
threshold is  0.564906
loss: 0.6394, acc: 0.5962
threshold is  0.56491214
loss: 0.6394, acc: 0.5962
threshold is  0.56482273
loss: 0.6394, acc: 0.5962
threshold is  

loss: 0.6381, acc: 0.5970
threshold is  0.56528866
loss: 0.6381, acc: 0.5970
threshold is  0.5652513
loss: 0.6381, acc: 0.5970
threshold is  0.5652443
loss: 0.6381, acc: 0.5970
threshold is  0.5648778
loss: 0.6381, acc: 0.5970
threshold is  0.56483686
loss: 0.6381, acc: 0.5970
threshold is  0.56485003
loss: 0.6381, acc: 0.5970
threshold is  0.5648306
loss: 0.6381, acc: 0.5970
threshold is  0.56474787
loss: 0.6380, acc: 0.5970
threshold is  0.56468797
loss: 0.6380, acc: 0.5970
threshold is  0.5646645
loss: 0.6380, acc: 0.5970
threshold is  0.564491
loss: 0.6380, acc: 0.5970
threshold is  0.5642873
loss: 0.6380, acc: 0.5970
threshold is  0.56443435
loss: 0.6380, acc: 0.5970
threshold is  0.5643581
loss: 0.6380, acc: 0.5970
threshold is  0.5642446
loss: 0.6380, acc: 0.5970
threshold is  0.5642613
loss: 0.6380, acc: 0.5970
threshold is  0.56419456
loss: 0.6380, acc: 0.5970
threshold is  0.5641455
loss: 0.6380, acc: 0.5970
threshold is  0.5641427
loss: 0.6380, acc: 0.5970
threshold is  0.56

loss: 0.6368, acc: 0.5980
threshold is  0.5670638
loss: 0.6368, acc: 0.5980
threshold is  0.5670317
loss: 0.6368, acc: 0.5980
threshold is  0.5671569
loss: 0.6368, acc: 0.5980
threshold is  0.56723815
loss: 0.6368, acc: 0.5980
threshold is  0.56707424
loss: 0.6368, acc: 0.5980
threshold is  0.5670258
loss: 0.6368, acc: 0.5980
threshold is  0.5670557
loss: 0.6368, acc: 0.5980
threshold is  0.5669362
loss: 0.6368, acc: 0.5980
threshold is  0.5669572
loss: 0.6368, acc: 0.5980
threshold is  0.5671158
loss: 0.6368, acc: 0.5980
threshold is  0.56720555
loss: 0.6368, acc: 0.5980
threshold is  0.5671291
loss: 0.6368, acc: 0.5980
threshold is  0.5672348
loss: 0.6368, acc: 0.5980
threshold is  0.567288
loss: 0.6368, acc: 0.5980
threshold is  0.5672009
loss: 0.6368, acc: 0.5980
threshold is  0.5672388
loss: 0.6368, acc: 0.5980
threshold is  0.5674658
loss: 0.6368, acc: 0.5980
threshold is  0.5674366
loss: 0.6368, acc: 0.5981
threshold is  0.5675122
loss: 0.6368, acc: 0.5981
threshold is  0.567589

loss: 0.6358, acc: 0.5985
threshold is  0.565288
loss: 0.6358, acc: 0.5985
threshold is  0.56521887
loss: 0.6358, acc: 0.5985
threshold is  0.56524277
loss: 0.6358, acc: 0.5985
threshold is  0.5652654
loss: 0.6358, acc: 0.5985
threshold is  0.56543857
loss: 0.6358, acc: 0.5985
threshold is  0.5652438
loss: 0.6358, acc: 0.5985
threshold is  0.5652424
loss: 0.6358, acc: 0.5985
threshold is  0.56543225
loss: 0.6358, acc: 0.5985
threshold is  0.56545067
loss: 0.6358, acc: 0.5986
threshold is  0.56545657
loss: 0.6358, acc: 0.5986
threshold is  0.5654695
loss: 0.6358, acc: 0.5986
threshold is  0.5655236
loss: 0.6358, acc: 0.5986
threshold is  0.56563884
loss: 0.6357, acc: 0.5986
threshold is  0.56582826
loss: 0.6357, acc: 0.5986
threshold is  0.5656826
loss: 0.6357, acc: 0.5986
threshold is  0.5655383
loss: 0.6357, acc: 0.5986
threshold is  0.56559616
loss: 0.6357, acc: 0.5986
threshold is  0.5657804
loss: 0.6357, acc: 0.5986
threshold is  0.5659476
loss: 0.6357, acc: 0.5986
threshold is  0.

In [ ]:
import matplotlib.pyplot as plt
mimicloss

In [ ]:
plt.scatter([i for i in range(len(mimicloss))],mimicloss)